In [32]:
import pandas as pd
from data_cleaning import clean_data
from data_scraping import get_data,extract_data, page_data

In [33]:
house_data = pd.read_csv("housing_data_full.csv",low_memory=False)
house_data = house_data.drop(house_data.index)

In [34]:
today = extract_data(get_data(0,1))

In [36]:
house_data

,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms,Type,Available From,Agency Commission,Rooms,Floor,Furnished,Main Window Facing,District,Area,Compound,Metro,Longtitude,Latitude,Agent,Description,First_post,Refresh,Balcony,Landlord lives in Shanghai,Oven,Recently renovated,Air Filter,English Speaking Landlord,Fitness Centers,Floor Heating,Garden,Historic Building,Large Storage Room,Parking,Pets Allowed\n\n \n false,Playground,Pool,Tennis Courts,Wall heating,Water Filter,Pets Allowed\n\n \n true,Good View,Yoga Facilities


In [20]:
house_data

,Listing_Id,Price,Size,N_Bedrooms,N_Bathrooms,Type,Available From,Agency Commission,Rooms,Floor,Furnished,Main Window Facing,District,Area,Compound,Metro,Longtitude,Latitude,Agent,Description,First_post,Refresh,Balcony,Landlord lives in Shanghai,Oven,Recently renovated,Air Filter,English Speaking Landlord,Fitness Centers,Floor Heating,Garden,Historic Building,Large Storage Room,Parking,Pets Allowed\n\n \n false,Playground,Pool,Tennis Courts,Wall heating,Water Filter,Pets Allowed\n\n \n true,Good View,Yoga Facilities


In [ ]:
house_data = pd.concat((house_data, today), ignore_index=True)
house_data["extra_index"] = house_data.Listing_Id
house_data.set_index("extra_index", inplace=True)
today = page_data(house_data)